In [1]:
#!python3 --version
#from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#print(sc.pythonVer)


In [2]:
#import os
#os.environ["PYSPARK_PYTHON"] = "python3"  # O "python" si ese es tu ejecutable


In [32]:
#!pip list

In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [34]:
#!pip uninstall -y pyspark
#pip uninstall -yspark

In [35]:
#!pip cache purge

#!pip install pyspark==3.5.2


In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
#Importar librerias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import glob
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os
import sys

In [5]:

file_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\CardReports\Cash_Advance_Transaction-20240903-0921.xls'

# Especificar la ruta del archivo Excel
output_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado2.xlsx'

# Especificar la ruta del archivo Excel
output_path3 = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado3.xlsx'

# Ruta de la carpeta que contiene los archivos
folder_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\CardReports'

In [98]:
# Obtener una lista de todos los archivos Excel en la carpeta
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.xls', '.xlsx'))]

dataframes = []
for file in file_list:
    df = pd.read_excel(file, sheet_name="Sheet2", skiprows=3, header=None)
    file_name = os.path.basename(file)
    new_columns = df.iloc[2].astype(str) + df.iloc[3].astype(str)
    df.columns = new_columns  # Asignar los nuevos encabezados
    df = df[4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
    df.reset_index(drop=True, inplace=True)
    df['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
    print("Nombre del archivo:", file_name)
    #Tranformación 1 - Nulos
    # Eliminar columnas donde todos los valores son NaN
    print ("Transformacion Nulos\n")
    df = df.dropna(axis=1, how='all')
    #Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
    df = df.fillna(0)
    df = df.loc[df["CS25DATE & TIME"] != "Totals:"]
    #print(df) 
    if "ATM" in file_name:
        print("El archivo contiene 'ATM' en el nombre.")
        df.insert(loc=df.columns.get_loc("nanSTATUS"), column="FEE($)", value=0)
        #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        df['DISPENSEDTOTAL($)'] = pd.to_numeric(df['DISPENSEDTOTAL($)'], errors='coerce').fillna(0).astype(float)
        df['TRANSAMOUNT($)'] = pd.to_numeric(df['TRANSAMOUNT($)'], errors='coerce').astype(float)
        df = df.drop(['DISPENSED QTYS$1', 'nan$5', 'nan$10', 'nan$20', 'nan$50', 'nan$100'], axis=1)
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSTYPE'] = df['TRANSTYPE'].astype(str)        
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['nanCARD#'] = df['nanCARD#'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSTYPE"] = df["TRANSTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSTYPE"] = df["TRANSTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS", "nanCARD#": "CARDNUMBER", "TRANSTYPE": "TRANSACTIONTYPE"})
        print(df) 
    elif "Cash_Advance" in file_name:
        print("El archivo contiene 'Cash_Advance' en el nombre.")  # Agregar el nombre del archivo
        #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")
        df['TRANSAMOUNT($)'] = pd.to_numeric(df['TRANSAMOUNT($)'], errors='coerce').astype(float)
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        # Filtrar los registros que no sean 'Credit'
        df = df.loc[df["TRANSACTIONTYPE"] != "Credit"]
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    elif "Debit_Dispense" in file_name:
        print("El archivo contiene 'Debit_Dispense' en el nombre.")  # Agregar el nombre del archivo  
          #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")       
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    elif "Pos_Ticket_Purchase" in file_name:
        print("El archivo contiene 'Pos_Ticket_Purchase' en el nombre.")  # Agregar el nombre del archivo  
          #Tranformación 2 - Limpia y Convertir las columnas numéricas a float
        status_column_index = df.columns.get_loc("FEE($)")       
        df.insert(loc=status_column_index, column="DISPENSEDTOTAL($)", value=0)
        df.insert(loc=status_column_index, column="COINSTOTAL($)", value=0)        
        #Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
        df['TRANSACTIONTYPE'] = df['TRANSACTIONTYPE'].astype(str)
        df['SEQUENCENUMBER'] = df['SEQUENCENUMBER'].astype(str)        
        df['CARDNUMBER'] = df['CARDNUMBER'].astype(str)        
        df['HOSTSEQ NUMBER'] = df['HOSTSEQ NUMBER'].astype(str)
        # limpiar la columna TRANSACTIONTYPE
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.strip()  # Elimina espacios antes y después
        df["TRANSACTIONTYPE"] = df["TRANSACTIONTYPE"].str.replace(r"\s+", " ", regex=True)  # Reemplaza múltiples espacios con uno solo
        df = df.rename(columns={"nanSTATUS": "STATUS"})
    else:
        print("otra Opcion") 
    df = df.rename(columns={"CS25DATE & TIME": "DATE & TIME"})
    dataframes.append(df)

df_combined = pd.concat(dataframes, ignore_index=True)

print("DataFrame combinado con nombres de archivo:")
print(df_combined)


#Código para concatenar filas 3 y 4 para los titulos 

#Lectura del archivo
#df = pd.read_excel(file_path, sheet_name="Sheet2", skiprows=3, header=None)
#file_name = os.path.basename(file_path)
#print("Nombre del archivo:", file_name)

#Antes de unificar
#print ("Titulos de archivo fuente:")
#print(df)

#Combinar filas 3 y 4 

#new_columns = df.iloc[2].astype(str) + df.iloc[3].astype(str)
#df.columns = new_columns  # Asignar los nuevos encabezados
#df = df[4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
# Reiniciar el índice
#df.reset_index(drop=True, inplace=True)

#print("Después de la unificación:")
#print(df)

#print(df.dtypes)

# Agregar la columna del nombre del archivo
#df["file_name"] = file_name

#Tranformación 1 - Nulos
# Eliminar columnas donde todos los valores son NaN
#print ("Transformacion Nulos\n")
#df = df.dropna(axis=1, how='all')
#print(df) 

#Tranformación 2 - Limpia y Convertir las columnas numéricas a float
#df['DISPENSEDTOTAL($)'] = pd.to_numeric(df['DISPENSEDTOTAL($)'], errors='coerce').fillna(0).astype(float)
#df['TRANSAMOUNT($)'] = pd.to_numeric(df['TRANSAMOUNT($)'], errors='coerce').astype(float)

#Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
#df = df.fillna(0)

#Tranformación 4-  Convierte las columnas categóricas o de texto a tipo `str`
#df['TRANSTYPE'] = df['TRANSTYPE'].astype(str)

#print(df)


#df = df.drop(['DISPENSED QTYS$1', 'nan$5', 'nan$10', 'nan$20', 'nan$50', 'nan$100'], axis=1)

# Guardar el DataFrame de Pandas en Excel
df_combined.to_excel(output_path, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path}")


Nombre del archivo: ATM_Transaction_Summary-20240903-0919.xls
Transformacion Nulos

El archivo contiene 'ATM' en el nombre.
          CS25DATE & TIME    SEQUENCENUMBER CARDNUMBER HOSTSEQ NUMBER  \
0     08/23/2024 08:13:19  2024082308124108       7816           1452   
1     08/23/2024 08:27:15  2024082308262908       9584           1453   
2     08/23/2024 08:46:09  2024082308453408       4299           1454   
3     08/23/2024 09:06:48  2024082309063308       2610              0   
4     08/23/2024 09:08:19  2024082309074608       6718           1456   
...                   ...               ...        ...            ...   
1194  08/31/2024 03:33:35  2024083103330508       6909           2872   
1195  08/31/2024 04:15:39  2024083104150508       7577           2874   
1196  08/31/2024 04:45:25  2024083104445708       6160              0   
1197  08/31/2024 04:48:35  2024083104480808       1284           2877   
1198  08/31/2024 05:10:02  2024083105092708       5484           2878   


C:\Users\paula\AppData\Local\Temp\ipykernel_73224\3348968049.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\paula\AppData\Local\Temp\ipykernel_73224\3348968049.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\paula\AppData\Local\Temp\ipykernel_73224\3348968049.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('fu

Archivo exportado exitosamente a C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado2.xlsx


In [19]:
#SPARK

#import os
#os.environ["PYSPARK_PYTHON"] = "python3"  # O "python" si ese es tu ejecutable
spark = SparkSession.builder.appName("PySparkExample").getOrCreate()
#spark.conf.set("spark.sql.shuffle.partitions", 1)  # Forzar a usar solo una partición

#Pandas a Spark

dfATM = spark.createDataFrame(df)

#Tranformación 1 - Nulos
# Eliminar columnas donde todos los valores son NaN
#print("Transformación Nulos\n")
#dfATM = dfATM.select([col_name for col_name in dfATM.columns if not dfATM.filter(col(col_name).isNotNull()).count() == 0])

#dfATM.printSchema()

# Convertir el DataFrame de Spark a Pandas
pandas_df = dfATM.toPandas()

# Imprimir el DataFrame de Pandas
print(pandas_df)



          CS25DATE & TIME    SEQUENCENUMBER nanCARD# HOSTSEQ NUMBER  \
0     08/23/2024 08:13:19  2024082308124108     7816           1452   
1     08/23/2024 08:27:15  2024082308262908     9584           1453   
2     08/23/2024 08:46:09  2024082308453408     4299           1454   
3     08/23/2024 09:06:48  2024082309063308     2610              0   
4     08/23/2024 09:08:19  2024082309074608     6718           1456   
...                   ...               ...      ...            ...   
1195  08/31/2024 04:15:39  2024083104150508     7577           2874   
1196  08/31/2024 04:45:25  2024083104445708     6160              0   
1197  08/31/2024 04:48:35  2024083104480808     1284           2877   
1198  08/31/2024 05:10:02  2024083105092708     5484           2878   
1199              Totals:                 0        0              0   

       TRANSTYPE  TRANSAMOUNT($) COINSTOTAL($)  DISPENSEDTOTAL($)  nanSTATUS  \
0     Withdrawal           100.0             0              100.0  

In [20]:
# Transformación 2: Filtros
# Filtro: DISPENSEDTOTAL($) > 0 y TRANSTYPE != 'Balance inquiry'
dfATM_Completadas = dfATM.filter((dfATM['DISPENSEDTOTAL($)'] > 0) & (dfATM['TRANSTYPE'] != 'Balance inquiry'))

dfATM_Completadas.show()

# Agregar columna con el nombre del archivo
#df_with_filename = dfATM.withColumn("file_name", file_name)

dfATM_Completadas_pd = dfATM_Completadas.toPandas()

# Guardar el DataFrame de Pandas en Excel
dfATM_Completadas_pd.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")


+-------------------+----------------+--------+--------------+----------+--------------+-------------+-----------------+---------+--------------------+
|    CS25DATE & TIME|  SEQUENCENUMBER|nanCARD#|HOSTSEQ NUMBER| TRANSTYPE|TRANSAMOUNT($)|COINSTOTAL($)|DISPENSEDTOTAL($)|nanSTATUS|           file_name|
+-------------------+----------------+--------+--------------+----------+--------------+-------------+-----------------+---------+--------------------+
|08/23/2024 08:13:19|2024082308124108|    7816|          1452|Withdrawal|         100.0|            0|            100.0|Completed|ATM_Transaction_S...|
|08/23/2024 08:27:15|2024082308262908|    9584|          1453|Withdrawal|          60.0|            0|             60.0|Completed|ATM_Transaction_S...|
|08/23/2024 08:46:09|2024082308453408|    4299|          1454|Withdrawal|         600.0|            0|            600.0|Completed|ATM_Transaction_S...|
|08/23/2024 09:08:19|2024082309074608|    6718|          1456|Withdrawal|        1000.0|